# Forecast Big Data 2025 — Pipeline Baseline

**Objetivo**: prever a quantidade semanal de vendas por `pdv`/`produto` para janeiro/2023 a partir do histórico de 2022.  
**Entregável**: `semana;pdv;produto;quantidade` (inteiros), CSV `;` UTF-8 ou Parquet.  
**Métrica de ranking**: WMAPE (quanto menor, melhor).  
**Observação**: parametrização para 4 ou 5 semanas de jan/2023.


In [ ]:
from pathlib import Path
import pandas as pd

# Caminhos locais (ajuste se necessário)
BASE_DIR  = Path(r"C:\Users\guede\OneDrive\Documentos\Hackathon\Bigdata\Arquivos\hackathon-forecast-2025")
DATA_PROC = BASE_DIR / "data" / "processed"

# Parâmetros do baseline
N_SEMANAS_MEDIA = 4   # janela de média das últimas N semanas do ano
N_SEMANAS_JAN   = 5   # 4 ou 5 semanas para janeiro/2023

pd.set_option("display.max_rows", 10)
pd.set_option("display.width", 120)


## Dados processados utilizados

- `data/processed/produtos.parquet` — dimensão de produtos (`produto`, `categoria`).
- `data/processed/pdvs.parquet` — dimensão de PDVs (`pdv`, `categoria_pdv`, `premise`, `zipcode`).
- `data/processed/transacoes_2022_diarias.parquet` — transações consolidadas por dia (`data;pdv;produto;quantidade`), 2022.


In [ ]:
df_prod = pd.read_parquet(DATA_PROC / "produtos.parquet")
df_pdv  = pd.read_parquet(DATA_PROC / "pdvs.parquet")
df      = pd.read_parquet(DATA_PROC / "transacoes_2022_diarias.parquet")

print("Shapes")
print(" - produtos          :", df_prod.shape)
print(" - pdvs              :", df_pdv.shape)
print(" - transações diárias:", df.shape)

display(df.head(5))


## Agregação semanal (ISO)

Agrega a série diária para semanal por `(pdv, produto, semana)` somando `quantidade` em 2022.


In [ ]:
df["ano"] = df["data"].dt.isocalendar().year
df["semana"] = df["data"].dt.isocalendar().week

df_sem = (df.query("ano == 2022")
            .groupby(["pdv","produto","semana"], as_index=False)["quantidade"]
            .sum())

df_sem.head(10)


## Baseline: média das últimas *N* semanas de 2022

Para cada `(pdv, produto)`, calcula-se a média de `quantidade` nas últimas `N_SEMANAS_MEDIA` semanas de 2022 e replica-se para `N_SEMANAS_JAN` semanas de jan/2023.


In [ ]:
# últimas N semanas de 2022
last_weeks = df_sem.query("semana > (52 - @N_SEMANAS_MEDIA)")

# média por (pdv, produto)
df_base = (last_weeks
           .groupby(["pdv","produto"], as_index=False)["quantidade"]
           .mean()
           .rename(columns={"quantidade":"qtd_prev"}))

# replicar para semanas de janeiro/2023
df_forecast = (
    pd.MultiIndex.from_product(
        [range(1, N_SEMANAS_JAN + 1), df_base["pdv"], df_base["produto"]],
        names=["semana","pdv","produto"]
    ).to_frame(index=False)
    .merge(df_base, on=["pdv","produto"], how="left")
)

# pós-processamento
df_forecast["quantidade"] = (df_forecast["qtd_prev"]
                             .round()
                             .clip(lower=0)
                             .astype(int))
df_forecast = df_forecast[["semana","pdv","produto","quantidade"]]

# export
out_csv     = DATA_PROC / "baseline_forecast.csv"
out_parquet = DATA_PROC / "baseline_forecast.parquet"
df_forecast.to_csv(out_csv, sep=";", index=False, encoding="utf-8")
df_forecast.to_parquet(out_parquet, index=False)

print("Arquivos salvos:")
print(" -", out_csv)
print(" -", out_parquet)
df_forecast.head(10)


## Checagens finais

- Colunas: `semana; pdv; produto; quantidade`  
- `quantidade` não negativa (inteiros)  
- CSV com separador `;` e UTF-8


In [ ]:
assert list(df_forecast.columns) == ["semana","pdv","produto","quantidade"]
assert df_forecast["quantidade"].min() >= 0
assert pd.api.types.is_integer_dtype(df_forecast["quantidade"])
print("OK — submissão formatada corretamente.")
print("shape:", df_forecast.shape)
